# Import thư viện

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from requests import get
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from time import time
from warnings import warn
from datetime import datetime
from datetime import timedelta
from pytz import timezone
from IPython.display import clear_output
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import re
import os
import sys
import glob
import shutil
from pathlib import Path
import re

In [2]:
# Mở trình duyệt Google Chrome ở trang thái Tab ẩn danh
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
s = Service('C:\chromedriver')
driver = webdriver.Chrome(service=s, options=options)
driver.implicitly_wait(0)

In [3]:
# List tất cả url movie
list_all_url_movie = pd.read_csv('E:/Crawl_Web/Rotten Tomatoes/Crawl URL/URL_movie.csv').squeeze('columns').tolist()

In [4]:
df_movie = []

for url_movie in list_all_url_movie[:1000]:
    # Mở url phim
    driver.get(url_movie)
    
    try:
        # Đợi cho đến khi url load xong
        element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="main_container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
        soup = BeautifulSoup(html_of_interest, 'lxml')

        sleep(2)

        ## Tạo Dict chứa thông tin về phim
        dict_info_movie = {}

        # Tiêu đề phim
        dict_info_movie['Title'] = soup.select('h1[class="scoreboard__title"]')[0].text

        # Điểm đánh giá của nhà phê bình
        dict_info_movie['Tomatometer score'] = soup.select('score-board')[0].attrs['tomatometerscore']

        # Điểm đánh giá của khán giả
        dict_info_movie['Audience score'] = soup.select('score-board')[0].attrs['audiencescore']

        # Số lượt đánh giá của nhà phê bình
        dict_info_movie['Tomatometer count'] = soup.select('a[slot="critics-count"]')[0].text.strip()

        # Số lượt đánh giá của khán giả
        dict_info_movie['Audience count'] = soup.select('a[slot="audience-count"]')[0].text.strip()

        # Trạng thái đánh giá phim của nhà phê bình
        dict_info_movie['Tomatometer state'] = soup.select('score-board')[0].attrs['tomatometerstate']

        # Trạng thái đánh giá phim của khán giả
        dict_info_movie['Audience state'] = soup.select('score-board')[0].attrs['audiencestate']

        # Thông tin phim
        list_name_info = soup.select('div[class="meta-label subtle"]')
        list_info = soup.select('div[data-qa="movie-info-item-value"]')

        for num_info in range(len(list_name_info)):
            name_info = list_name_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
            info = list_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
            dict_info_movie[name_info] = info

        # Tóm tắt nội dụng phim
        dict_info_movie['Synopsis'] = soup.select('div[id="movieSynopsis"]')[0].text.strip()

        # Diễn viên
        list_raw_cast = soup.select('span[class="characters subtle smaller"]')
        list_cast = []

        for c in list_raw_cast:
            list_cast.append(c.attrs['title'])

        dict_info_movie['Cast'] = ', '.join(list_cast)

        df_movie.append(dict_info_movie)

        # Reviews của nhà phê bình
        driver.get(url_movie + '/reviews')

        element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="layout reviews-page-container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
        soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

        ## Check có review của nhà phê bình không ?
        check_critic_review = soup_critic_review.select('div[class="review_table"]')[0].text.strip()

        if check_critic_review != '':
            list_critic_review = []
            
            for rv in soup_critic_review.select('p[class="review-text"]'):
                list_critic_review.append(rv.text)
            
            while True:
                sleep(1)
                try:
                    next_button = driver.find_element_by_css_selector('rt-button[class="js-prev-next-paging-next"]')
                    driver.execute_script("arguments[0].click();", next_button)
                    
                    element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="review_table"]')))
                    html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
                    soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

                    for rv in soup_critic_review.select('p[class="review-text"]'):
                        list_critic_review.append(rv.text.replace('  ', ''))
                    
                except:
                    break
            dict_info_movie['Critic review'] = '<>'.join(list_critic_review)
        elif check_critic_review == '':
            dict_info_movie['Critic review'] = 'Không có review của nhà phê bình'

        # Reviews của khán giả
        driver.get(url_movie + '/reviews?type=user')

        element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-qa="reviews-container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
        soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

        ## Check có review của khán giả không ?
        check_audience_review = soup_audience_review.select('div[id="movieUserReviewsContent"]')

        if len(check_audience_review) != 0:
            list_audience_review = []
            
            for rv in soup_audience_review.select('p[data-qa="review-text"]'):
                list_audience_review.append(rv.text)
            
            while True:
                sleep(1)
                try:
                    next_button = driver.find_element_by_css_selector('rt-button[class="js-prev-next-paging-next"]')
                    driver.execute_script("arguments[0].click();", next_button)
                    
                    element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="movieUserReviewsContent"]')))
                    html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
                    soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

                    for rv in soup_audience_review.select('p[data-qa="review-text"]'):
                        list_audience_review.append(rv.text)
                    
                except:
                    break
            dict_info_movie['Audience review'] = '<>'.join(list_audience_review)
        else:
            dict_info_movie['Audience review'] = 'Không có review của khán giả'

    except:
        print(url_movie)
        continue
    
df_movie = pd.DataFrame(df_movie)

https://www.rottentomatoes.com/m/fury-river
https://www.rottentomatoes.com/m/children_of_the_corn_revelation
https://www.rottentomatoes.com/m/exhibition_on_screen_the_artists_garden_american_impressionism_2017
https://www.rottentomatoes.com/m/roja
https://www.rottentomatoes.com/m/goodbye_christopher_robin
https://www.rottentomatoes.com/m/tomorrow-morning-sutra-ujutru
https://www.rottentomatoes.com/m/beauty-and-the-beast-the-enchanted-christmas
https://www.rottentomatoes.com/m/sasaki_in_my_mind
https://www.rottentomatoes.com/m/kurt_vonnegut_unstuck_in_time


In [5]:
df_movie.to_csv('Crawl_test_1000_movie.csv', index=False)